In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
import math
import matplotlib.pyplot as plt

In [2]:
str_files = ''

#df = pd.read_csv(str_files + "ph.csv",encoding='utf-8',sep=';')
dateparse = lambda dates:pd.datetime.strptime(dates,'%d/%m/%Y')
df = pd.read_csv(str_files + 'dbo.CSV',encoding='utf-8',sep=';', parse_dates = ['data_coleta'],index_col=['data_coleta'],date_parser=dateparse)

In [3]:
z = df.loc[df['UGRHI'] == '10 - SOROCABA/MEDIO TIETE']

In [4]:
z = z.dropna(axis=1,how='any')

In [5]:
trainingd = z.resample('3M').mean()
trainingd
data = trainingd.iloc[:,1:2].values
train = trainingd.iloc[0:132,1:2].values  
test = trainingd.iloc[132:,1:2].values   
print('treino:',len(train))
print('teste:',len(test))

treino: 132
teste: 34


In [6]:
sc = MinMaxScaler()
train = sc.fit_transform(train)
test = sc.fit_transform(test)

In [7]:
def prepare_data(data, lags):
    X, y = [], []
    for row in range(len(data) - lags - 1):
        a = data[row:(row + lags), 0]
        X.append(a)
        y.append(data[row + lags, 0])
    return np.array(X), np.array(y)  

In [8]:
lags = 1
train_X, train_y = prepare_data(train, lags)
test_X, test_y = prepare_data(test, lags)

train_X = np.reshape(train_X, (train_X.shape[0],1,train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0],1,test_X.shape[1]))
print('Novo formato de treino:')
print(train_X.shape) 

Novo formato de treino:
(130, 1, 1)


In [9]:
model = Sequential() 

model.add(LSTM(units = 10, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1,patience=5)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 18s 135ms/sample - loss: 0.2082 - val_loss: 0.1636
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.2001 - val_loss: 0.1570
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1914 - val_loss: 0.1502
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1824 - val_loss: 0.1434
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1743 - val_loss: 0.1367
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1658 - val_loss: 0.1303
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1576 - val_loss: 0.1239
Epoch 8/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.1498 - val_loss: 0.1176
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1403 - val_loss: 0.1112
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 3ms/sample - loss: 0.0419 - val_loss: 0.0581
Epoch 80/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0398 - val_loss: 0.0582
Epoch 81/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0403 - val_loss: 0.0583
Epoch 82/100
130/130 [==============================] - 1s 4ms/sample - loss: 0.0404 - val_loss: 0.0583
Epoch 83/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0439 - val_loss: 0.0577
Epoch 84/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0416 - val_loss: 0.0574
Epoch 85/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0455 - val_loss: 0.0577
Epoch 86/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0398 - val_loss: 0.0582
Epoch 87/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0430 - val_loss: 0.0584
Epoch 88/100
130/130 [==============================] - 0s 1ms/sample - loss:

In [10]:
train_score10 = model.evaluate(train_X, train_y, verbose=0)
print(train_score10)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score10, math.sqrt(train_score10)))
test_score10 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score10, math.sqrt(test_score10)))

0.03530455111311032
Pontuação de Treino: 0.04 MSE (0.19 RMSE)
Pontuação de teste: 0.06 MSE (0.24 RMSE)


In [11]:
model = Sequential() 

model.add(LSTM(units = 25, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='min',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 17s 132ms/sample - loss: 0.2075 - val_loss: 0.1615
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1960 - val_loss: 0.1523
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1847 - val_loss: 0.1430
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1726 - val_loss: 0.1333
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1604 - val_loss: 0.1233
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1467 - val_loss: 0.1131
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1340 - val_loss: 0.1029
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1202 - val_loss: 0.0925
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1030 - val_loss: 0.0823
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0387 - val_loss: 0.0558
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0368 - val_loss: 0.0562
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0380 - val_loss: 0.0575
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0336 - val_loss: 0.0601
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0331 - val_loss: 0.0623
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0358 - val_loss: 0.0622
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0343 - val_loss: 0.0609
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0347 - val_loss: 0.0609
Epoch 87/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0399 - val_loss: 0.0595
Epoch 88/100
130/130 [==============================] - 0s 1ms/sample - loss:

In [12]:
train_score25 = model.evaluate(train_X, train_y, verbose=0)
print(train_score25)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score25, math.sqrt(train_score25)))
test_score25 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score25, math.sqrt(test_score25)))

0.03262596124639878
Pontuação de Treino: 0.03 MSE (0.18 RMSE)
Pontuação de teste: 0.06 MSE (0.24 RMSE)


In [13]:
model = Sequential() 

model.add(LSTM(units = 50, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 17s 130ms/sample - loss: 0.2064 - val_loss: 0.1578
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1900 - val_loss: 0.1446
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1728 - val_loss: 0.1307
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1549 - val_loss: 0.1160
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1348 - val_loss: 0.0998
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.1130 - val_loss: 0.0835
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0887 - val_loss: 0.0698
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0659 - val_loss: 0.0607
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0501 - val_loss: 0.0599
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0357 - val_loss: 0.0612
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0347 - val_loss: 0.0590
Epoch 81/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0355 - val_loss: 0.0583
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0333 - val_loss: 0.0580
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0346 - val_loss: 0.0592
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0327 - val_loss: 0.0607
Epoch 85/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0329 - val_loss: 0.0604
Epoch 86/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0327 - val_loss: 0.0600
Epoch 87/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0362 - val_loss: 0.0607
Epoch 88/100
130/130 [==============================] - 0s 2ms/sample - loss:

In [14]:
train_score50 = model.evaluate(train_X, train_y, verbose=0)
print(train_score50)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score50, math.sqrt(train_score50)))
test_score50 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score50, math.sqrt(test_score50)))

0.03210866474188291
Pontuação de Treino: 0.03 MSE (0.18 RMSE)
Pontuação de teste: 0.06 MSE (0.24 RMSE)
